In [1]:
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()
import sys
sys.path.append(r'~/Pose Estimation Keras')
import cnn
from cnn import conv_base
from build_dataset import Dataset
import numpy as np
# from sklearn.preprocessing import normalize
from scipy import stats

from keras import layers
from keras import models
from keras import optimizers
import keras.backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler

from time import time
import cv2
import os 


Using TensorFlow backend.
/home/bhandawatlab_duke/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [2]:
total_frames = 5
frame_height = 224
frame_width = 224
frame_channels = 3
num_bodyparts = 6




def build_deconv_model():
    frame_tensor = conv_base.layers[0].get_input_at(0)
    resnet_output = cnn.convolution_network(frame_tensor)
    model_resnet = models.Model(inputs=[frame_tensor], outputs=[resnet_output])
    
    deconv_output = cnn.deconvolution_fusion_network(model_resnet.output)
    deconv_separate_output = cnn.separation_network(deconv_output)
    model_deconv = models.Model(inputs=[frame_tensor], outputs=[deconv_output , deconv_separate_output])
    
    losses = {"deconvolution_network_output": "binary_crossentropy",
        "deconvolution_fusion_network_output": "binary_crossentropy",
        
#         "spatial_fusion_output": "binary_crossentropy",
#         "optical_flow_output": "binary_crossentropy"
    }
    lossWeights = {"deconvolution_fusion_network_output": 1.0,
                  "deconvolution_network_output": 1.0}
#                    "spatial_fusion_output": 1.0,
#                    "optical_flow_output": 1.0}
    
#     opt = optimizers.SGD(lr = 0.0, momentum=0.9, nesterov=True)
    opt = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    model_deconv.compile( optimizer=opt, loss=losses, loss_weights=lossWeights, metrics = ["accuracy"])
    
    return model_deconv

model = build_deconv_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 30)        552990    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 24)        6504      
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 56, 56, 18)        3906      
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 112, 112, 12)      1956      
_________________________________________________________________
same (Conv2DTranspose)       (None, 224, 224, 6)       654       
__________

In [3]:
import csv
def parse_csv(file):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        indices = []
        for line in csv_reader:
            indices.extend([int(f) for f in line if f!=''])
    return indices

indices = parse_csv('dataset_indices.csv')
print(indices)

[0, 2, 3, 4, 6, 7, 8, 11, 15, 16, 17, 18, 19, 20, 23, 120, 126, 127, 178, 185, 199, 228, 244, 261, 267, 285, 306, 335, 367, 375, 389, 405, 410, 415, 420, 426, 430, 436, 440, 445, 452, 455, 460, 465, 470, 476, 480, 485, 490, 496, 500, 511, 520, 525, 530, 535, 542, 552, 555, 568, 570, 575, 581, 588, 595, 600, 605, 610, 615, 621, 625, 633, 636, 640, 645, 650, 655, 665, 670, 675, 680, 685, 690, 695, 700, 705, 710, 715, 718, 723, 725, 751, 755, 770, 783, 784, 804, 812, 816, 823, 829, 838, 841, 845, 850, 855, 863, 865, 891, 894, 904, 908, 915, 920, 929, 946, 953, 955, 959, 965, 970, 980, 987, 990, 995, 1030, 1035, 1041, 1045, 1160, 1164, 1170, 1174, 1220, 1225, 1230, 1235, 1240, 1245, 1250, 1255, 1267, 1275, 1280, 1285, 1290]


In [4]:
import h5py
with h5py.File('../dataset.h5', 'r') as hf:
#     central_heatmap = hf['central_heatmap'][indices]
    frames = hf['frames'][indices]
    heatmaps_individual = hf['heatmaps_individual'][indices]
    heatmaps_combined = hf['heatmaps_combined'][indices]
    
home_dir = os.getcwd()
# central_heatmap = np.expand_dims(central_heatmap, axis=-1)
heatmaps_combined = np.expand_dims(heatmaps_combined, axis=-1)
heatmaps_individual = np.swapaxes(heatmaps_individual, 2, 4)
# print(central_heatmap.shape)
print(heatmaps_individual.shape)
print(heatmaps_combined.shape)

(146, 5, 224, 224, 6)
(146, 5, 224, 224, 1)


In [5]:
def step_decay(epoch):
    try:
        if epoch < 30:
            lrate = 0.0001
        elif epoch < 70:
            lrate = 0.00005
        elif epoch < 200:
            lrate = 0.00001
        return lrate
    except:
        print('Error in setting learning rate')
lrate = LearningRateScheduler(step_decay, verbose=1)

checkpointer = ModelCheckpoint(filepath= os.path.join(home_dir, "pretrained_models/model_chained_fusion_11419.h5"),
                               verbose=1,
                               monitor='loss')

tensorboard = TensorBoard(log_dir=os.path.join(home_dir, "logs/{}".format(time())))

In [6]:
def normalize(x):
    x = np.array(x, dtype = np.float64)
    x -= np.mean(x, dtype = np.float64)
    x /= np.std(x, dtype = np.float64)
    return x


def generator(frames, heatmaps_individual, heatmaps_combined):
    while(1):
        for j,l,m in zip(frames, heatmaps_individual, heatmaps_combined):
#             print(l.shape[0])
            for channel in range(l.shape[0]):
                l[channel, :, :, :] = normalize(l[channel, :, :, :]) 
                m[channel, :, :, :] = normalize(m[channel, :, :, :])
            # include batch dimension and repeat the matrix at axis 0 num_frames times to match the target dimensionality
            yield (j, {
#                 "optical_flow_output" : np.repeat(np.expand_dims(k, axis=0), 5, 0), 
                "deconvolution_network_output": l,
                "deconvolution_fusion_network_output": m}) 
            
model.fit_generator(generator(frames,heatmaps_individual, heatmaps_combined),
                           steps_per_epoch=100, 
                           verbose = 1,
                           epochs = 150,
                           callbacks=[lrate, checkpointer, tensorboard])

Epoch 1/150

Epoch 00001: LearningRateScheduler setting learning rate to 0.0001.
100/100 [==============================] - 52s 516ms/step - loss: 0.7958 - deconvolution_fusion_network_output_loss: 0.3061 - deconvolution_network_output_loss: 0.4742 - deconvolution_fusion_network_output_acc: 0.9961 - deconvolution_network_output_acc: 0.8517

Epoch 00001: saving model to /home/bhandawatlab_duke/Pose-Estimation-Keras/pretrained_models/model_chained_fusion_11419.h5
Epoch 2/150

Epoch 00002: LearningRateScheduler setting learning rate to 0.0001.
100/100 [==============================] - 38s 382ms/step - loss: 0.4593 - deconvolution_fusion_network_output_loss: 0.1826 - deconvolution_network_output_loss: 0.2611 - deconvolution_fusion_network_output_acc: 0.9962 - deconvolution_network_output_acc: 0.9994

Epoch 00002: saving model to /home/bhandawatlab_duke/Pose-Estimation-Keras/pretrained_models/model_chained_fusion_11419.h5
Epoch 3/150

Epoch 00003: LearningRateScheduler setting learning rat